In [1]:
import pprint, pickle
import numpy as np
import SimpleITK as sitk
import os

In [2]:
def save_pickle(obj, file: str, mode: str = 'wb') -> None:
    with open(file, mode) as f:
        pickle.dump(obj, f)

In [3]:
def maybe_create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [18]:
head_cut_pred_folder = '/home/ubuntu/codes/radiology/3nnUnet/nnUNet_output/nnUNet_trained_models/nnUNet/3d_fullres/Task152_CellPress110/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/Task174_CellPressTest152'
full_save_foder = os.path.join(head_cut_pred_folder,'full')
maybe_create_path(full_save_foder)
pred_suffix = '.nii.gz'

In [19]:
pkl_file = open('/home/ubuntu/codes/radiology/file/after_headcut_properties_CellPressTest.pkl', 'rb')
data1 = pickle.load(pkl_file)
pprint.pprint(len(data1))
#pprint.pprint(data1)
pkl_file.close()

152


In [21]:
for key in data1:
    XJTsN_name = key
    if key == 'ExtA0009':
        continue
    properti = data1[XJTsN_name]
    min_z, max_z, min_y, max_y, min_x, max_x = properti['coords']
    
    if os.path.exists(os.path.join(head_cut_pred_folder,XJTsN_name + pred_suffix)):
        if os.path.exists(os.path.join(full_save_foder,XJTsN_name+'_full.nii.gz')):
            #print ('./full/'+XJTsN_name+'_full.nii.gz','exists')
            continue
        pred_img_nii = sitk.ReadImage(os.path.join(head_cut_pred_folder,XJTsN_name + pred_suffix))
        pred_img = sitk.GetArrayFromImage(pred_img_nii).astype(np.int32)

        prediction_instance_shape = properti['before_size']
        prediction = np.zeros(prediction_instance_shape, dtype=np.int32)
        try:
            prediction[min_z:max_z, min_y:max_y, min_x:max_x] =pred_img
        except:
            print ('somethong wrong with ',XJTsN_name)
            print (pred_img.shape,prediction.shape)
            print (properti)

        pred_itk_image = sitk.GetImageFromArray(prediction)
        pred_itk_image.SetSpacing(properti["itk_spacing"])
        pred_itk_image.SetOrigin(properti["itk_origin"])

        sitk.WriteImage(pred_itk_image, os.path.join(full_save_foder,XJTsN_name+'_full.nii.gz'))

somethong wrong with  Ts0001
